<a id="top"></a>
# Benchmark Models on Intel® DevCloud for the Edge

## Intel® Distribution of OpenVINO™ Toolkit version check:
You are currently using the latest development version of Intel® Distribution of OpenVINO™ Toolkit.

### Introduction
This notebook uses the [Intel® Distribution of OpenVINO™ Toolkit Benchmarking app](https://docs.openvinotoolkit.org/latest/_inference_engine_tools_benchmark_tool_README.html) to benchmark your on different hardware. Also, showcasing how you can optionally store your model files using AWS S3 service.

Benchmark python tool provides estimation of deep learning inference performance on the supported devices. Performance can be measured for two inference modes: synchronous (latency-oriented) and asynchronous (throughput-oriented).  

This tutorial benchmarks the deep learning model with 
1. Intel® Core™ CPU, Intel® Xeon® CPU, Intel® HD Graphics 530 GPU, Intel® Neural Compute Stick 2, Intel® Movidius™ Myriad™ X HDDL
2. Workload distribution with [Multi plugin](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_MULTI.html)  

#### Running this notebook without  AWS S3 bucket
If you are not using AWS S3 service to store model files, please read the instructions below before running this notebook.
1. Upload model files (IR files (.xml+.bin)) to your Intel® DevCloud instance inside the same directory as that of jupyter notebook. <br>Folder structure should be same format as below:
    <br> 
    
    *[Model_Name]/IR_models/[FP16 or FP32]/[IR FILE NAME].xml* 
    
    
2. Initialize "models" list with name of models you have uploaded for benchmarking. [See section 1.9 - Creating list of FP16 and FP32 models](#Creating-lists-of-FP16-and-FP32-models)<br>
    Example: *models = ['MobileNetV2-keras', 'NASNetMobile-keras']*<br>


3. Skip **section 1.4 - Enter AWS S3 bucket name** and **section 1.8 - Downloading Model from S3**

### How It Works
Upon start-up, the application reads command-line parameters and loads a network and images/binary files to the Inference Engine plugin, which is chosen depending on a specified device. The number of infer requests and execution approach depend on the mode defined with the -api and -niter command-line parameter. 

In this tutorial, we use following input parameters with the benchmark app:

- -m: deep learning model to infer in Intermediate Representation (.xml & .bin), e.g. Resnet-50 tensorflow model. 
- -d: device to offload inference workload
- -niter: number of iterations. 
- -api: sync or async
- --report_type: information about details counter e.g. FPS and latency
- --report_folder: Path to a folder where statistics report is stored.
- -i: input image/video, If a topology is not data sensitive, you can skip the input parameter. 



### Enter AWS IAM user credentials

Provide AWS IAM user credentials (AccessKeyId,SecretAccessKey) to configure AWS SDK. 
1. Sign in to [IAM Console](https://console.aws.amazon.com/iam/home#/home)
2. Create a new IAM User with programmatic access 
3. Attach 'AmazonS3FullAccess' & 'AmazonSESFullAccess' policy to IAM User
4. Create Access Key - Please download 'Download.csv' file for future reference. AWS Access keys are used to configure AWS CLI.  


<div class=note><i><b>Note: </b>For detailed instructions, Please follow this <a href="https://docs.aws.amazon.com/IAM/latest/UserGuide/id_users_create.html#id_users_create_console">link</a></i></div>

In [ ]:
# Provide AWS IAM user credentials (AccessKeyId,SecretAccessKey)
ACCESS_KEY_ID = "AWS AccessKeyId"
SECRET_ACCESS_KEY = "AWS SecretAccessKey"

### Enter AWS S3 bucket name

Provide the AWS S3 bucket name that contains OpenVINO optimized model files (IR Files) in zip format. 

**Format**: [Model_Name].zip   
**Folder structure after unzip**: [Model_Name]/IR_models/[FP16 or FP32]/[IR FILE NAME].xml  
**Example**: MobileNetV2-keras.zip  
**Folder structure example**: MobileNetV2-keras/IR_models/FP16/MobileNetV2-keras.xml<br><br>
<div class=note><i><b>Note: </b>Do not place any other files or folders in this bucket</i></div><br>
<div class=tip><b>Tip: </b>Skip this section if you are not using AWS S3 bucket to store model files</div>


In [ ]:
BUCKET_NAME = "S3 Bucket Name"

### Enter Email address to send notification

Provide AWS SES verfied email address of sender and recipient. You can also provide same email address for both sender and reciever.  
For Email verification procedure, refer [here](https://docs.aws.amazon.com/ses/latest/DeveloperGuide/verify-email-addresses-procedure.html).<br><br>
<div class=tip><b>Tip: </b>Update AWS_REGION with the AWS region where you have verfied email addreses.</div>

In [ ]:
# Replace sender@example.com with your "From" address.
# This address must be verified with Amazon SES.
SENDER = "Email Address of Sender"

# Replace recipient@example.com with a "To" address. If your account 
# is still in the sandbox, this address must be verified.
RECIPIENT = "Email Address of Recipient"

# Replace us-west-2 with the AWS Region you're using for Amazon SES.
AWS_REGION = "us-west-2"

###  Install boto3 & progress package

In [ ]:
!pip3 install --upgrade pip
!pip3 install -r requirements.txt 

### Importing Packages

In [ ]:
from IPython.display import HTML
import os
import time
import pandas as pd


<div class=tip><b>Troubleshooting Tip: </b> If you have faced any issues in importing packages, please restart the kernel and run the jupyter notebook again</div>


### Downloading Model from S3

Download all the files from the S3 bucket to the Intel® DevCloud instance<br><br>
<div class=tip><b>Tip: </b>Skip this section if you are not using AWS S3 bucket to store model files</div>

In [ ]:
import boto3
import os
from zipfile import ZipFile
from botocore.exceptions import ClientError

# initializing list. 
models=[]

try:
    # initiate s3 resource
    s3 = boto3.resource('s3',aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)
    
    # select bucket
    my_bucket = s3.Bucket(BUCKET_NAME)

    # download file into current directory
    for s3_object in my_bucket.objects.all():
        # Need to split s3_object.key into path and file name, else it will give error file not found.
        path, file = os.path.split(s3_object.key)
        if(file):
            my_bucket.download_file(Key=s3_object.key,Filename=file)
            models.append(file.split(".")[0])
            zip_file = ZipFile(file)
            zip_file.extractall()
            
except ClientError as e:
    print(e.response['Error']['Message'])

if len(models) == 0:
    print("S3 bucket is empty. Please uplaod files to S3 bucket")
else: 
    print("List of available models:")
    print(models)

### Creating lists of FP16 and FP32 models

<div class=tip><b>Tip: </b>If you want to manually provide list of models, initialize "models" list with name of models you wish to benchmark.<br>
For example: models = ['MobileNetV2-keras', 'NASNetMobile-keras']</div>

In [ ]:
# Uncomment below line if you want to manually provide list of models. For example: models = ['MobileNetV2-keras', 'NASNetMobile-keras']
# models = [list of model separated by commas] 

models_fp32 = []
models_fp16 = []
for model in models:
    if 'FP32' in os.listdir(model+'/IR_models'):
        models_fp32.append(model)
    elif 'FP16' in os.listdir(model+'/IR_models'): 
        models_fp16.append(model)
print("FP32 Models:",models_fp32)
print("FP16 Models:",models_fp16)

### Creating job file
Till now, we ran all the above steps on a single development node with one core allocated for your account. Now we want to run inference on different edge compute nodes on the Intel® DevCloud for the Edge to benchmark the inference performance. For that, we will submit the inference jobs for each edge device in a queue. For each job, we will specify the type of the edge compute node that must be allocated for the job.

The job file in the below cell is written in Bash, and will be executed directly on the edge compute node. Run the following cell to write this in to the file "benchmark_app_job.sh"

<div class=note><i><b>Note: </b>Default model path is [Model Name]/IR_models/[FP16 or FP32]/[IR FILE NAME].xml.</i></div>

In [ ]:
%%writefile benchmark_app_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR
JOB_ID=`basename ${0} | cut -f1 -d"."`
OUTPUT_FILE=$1
DEVICE=$2
#FP_MODEL=$3
API=$3

# Provide the model path
# Argument 4 will be model name
# Update the .xml path(relative path) if required 

MODEL_DIR=$4/IR_models/$5

MODEL_FILE=$(ls $PBS_O_WORKDIR/${MODEL_DIR} | grep .xml)
MODEL_PATH=${MODEL_DIR}/${MODEL_FILE}

# Benchmark Application script writes output to a file inside a directory. We make sure that this directory exists.
#  The output directory is the first argument of the bash script
mkdir -p $OUTPUT_FILE

SAMPLEPATH=$PBS_O_WORKDIR

rm -f ${OUTPUT_FILE}/*

echo ${SAMPLEPATH}/${OUTPUT_FILE} > benchmark_filename_${JOB_ID}.txt

# Running the benchmark application code

python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py -m ${SAMPLEPATH}/${MODEL_PATH} \
            -d $DEVICE \
            -niter 10 \
            -api $API \
            --report_type detailed_counters \
            --report_folder ${SAMPLEPATH}/${OUTPUT_FILE}

Now that we have the job script, we can submit the jobs to edge compute nodes. In the IoT DevCloud, you can do this using the qsub command. We can submit object_detection_job to 5 different types of edge compute nodes simultaneously or just one node at at time.

There are three options of qsub command that we use for this:

-l : this option lets us select the number and the type of nodes using nodes={node_count}:{property}.

-F : this option lets us send arguments to the bash script.

-N : this option lets use name the job so that it is easier to distinguish between them.
If you are curious to see the available types of nodes on the IoT DevCloud, run the following optional cell.

In [ ]:
import subprocess
node_list = []
nodes = subprocess.check_output("pbsnodes | grep compnode | sort | uniq | awk ' /'properties'/  {print $3}'",shell = True)
nodes = nodes.decode('utf-8')
nodes = nodes.split('\n')
for node in nodes:
    node_list.append(node.split(',')[0])

## Wait until the benchmarking report files are written 

We submit the job to different hardware platforms using a job queue. We will have to wait until we get the results back from our specified hardware. In the following script, we check if the reports file is generated that shows the job is complete. Until, the job is completed, we print dots on the screen

In [ ]:
def wait_for_job_to_finish(job_id,verbose,model):
    # time-out set to 10 mins
    time_out = 10
    start_time = time.time()
    # print(job_id[0]) 
    if job_id:
        
        print("Job submitted to the queue. Waiting for it to complete .", end="")
        filename = "benchmark_filename_{}.txt".format(job_id[0].split(".")[0])
        
        while not os.path.exists(filename):  # Wait until the file report is created.
            time.sleep(1)
            print(".", end="")

        # Print the results
        with open(filename) as f:
            results_dir = f.read().split("\n")[0]
            
        report_filename = os.path.join(results_dir, "benchmark_report.csv") # Wait until the file report is created.
        while not os.path.exists(report_filename):
            time.sleep(1)
            print(".", end="")
            current_time = time.time()
            seconds_elapsed = current_time - start_time
            minutes = seconds_elapsed / 60.0 
            if(minutes >= time_out):
                print(" Time out: Exceeded maximum time of 10 mins !!!")
                return {
                        "Throughput (FPS)": 0.0, 
                        "Load network time (ms)" : 0.0,
                        "Read network time (ms)" : 0.0,
                        "First inference time (ms)": 0.0,
                        "Total execution time (ms)": 0.0,
                        "Total number of iterations": 0.0}
        
        df = pd.read_csv(report_filename, delimiter=";")
        
        if(verbose):
            print(df)
        
        throughput = float(df.loc["throughput"][0])
        device = df.loc["target device"][0]
        load_time = float(df.loc["load network time (ms)"][0])
        read_time = float(df.loc["read network time (ms)"][0])
        infer_time = float(df.loc["first inference time (ms)"][0])
        exec_time = float(df.loc["total execution time (ms)"][0])
        iterations = int(df.loc["total number of iterations"][0])
        
        os.remove(filename) # Cleanup
        print("\n===Benchmarking of {} model completed===\n".format(model))
        
    else:
        print("Error in job submission.")
        
        throughput = None
        device = None
        load_time = None
        read_time = None
        
    return {
            "Throughput (FPS)": throughput, 
            "Load network time (ms)" : load_time,
            "Read network time (ms)" : read_time,
            "First inference time (ms)": infer_time,
            "Total execution time (ms)": exec_time,
            "Total number of iterations": iterations}

The above wait_for_job_to_finish() function returns throughput, load network time and read network time. We save these return values in a dictionary called benchmarks to be used later in graphs

In [ ]:
# initialize output dictionary 
output = {} # Save benchmark reports of all model into a dict
for model in models:
    output[model]={}

## Benchmark Individual system with the deep learning model
The Multi-Device plugin automatically assigns inference requests to available computational devices to execute the requests in parallel. 

This example shows how to use MULTI plugin from the Intel® Distribution of OpenVINO™ toolkit.
First, let's take a look at the model's inference performance on each single device.

<div class=note><i><b>Note: </b> Set verbose flag to 'True' for printing intermediate benchmark outputs from each node</i></div>

In [ ]:
verbose = False

#### Run the Benchmark tool app with Intel® Core™ CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870</a> edge node with an <a 
    href="https://devcloud.intel.com/edge/devices/intel-core-i5-6500te-cpu/">Intel® Core™ i5 6500TE CPU</a>. The inference workload will run on the CPU.

In [ ]:
node = 'idc001skl'
if node in node_list:
    print("Submitting a job to an edge compute node with an Intel Core CPU...")
    for model in models_fp16:
        benchmarks = {}
        try:
            # Submit job to the queue
            job_id_core = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/core/{model} CPU async {model} FP16"
            benchmarks['Intel® Core™ CPU'] = wait_for_job_to_finish(job_id_core,verbose,model)
            output[model].update(benchmarks)
        except Exception as e:
            print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
            print("Error: ", e)

    for model in models_fp32:
        benchmarks = {}
        try:
            # Submit job to the queue
            job_id_core = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/core/{model} CPU async {model} FP32"
            benchmarks['Intel® Core™ CPU'] = wait_for_job_to_finish(job_id_core,verbose,model)
            output[model].update(benchmarks)
        except Exception as e:
            print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
            print("Error: ", e)
else:
    print(" {} not currently available. Try changing node !!! ".format(node))
    print("List of available nodes: {}".format(node_list))


#### Run the Benchmark tool app with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://devcloud.intel.com/edge/devices/intel-xeon-e3-1268l-v5-cpu/">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
node = 'idc007xv5'
if node in node_list:
    print("Submitting a job to an edge compute node with an Intel Xeon CPU...")
    for model in models_fp16:
        benchmarks={}
        try:
            # Submit job to the queue
            job_id_xeon = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/xeon/{model} CPU async {model} FP16"      
            benchmarks["Intel® Xeon® CPU"] = wait_for_job_to_finish(job_id_xeon,verbose,model)
            output[model].update(benchmarks)
        except Exception as e:
            print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
            print("Error: ", e)

    for model in models_fp32:
        benchmarks={}
        try:
            # Submit job to the queue
            job_id_xeon = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/xeon/{model} CPU async {model} FP32"      
            benchmarks["Intel® Xeon® CPU"] = wait_for_job_to_finish(job_id_xeon,verbose,model)
            output[model].update(benchmarks)
        except Exception as e:
            print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
            print("Error: ", e)
else:
    print(" {} not currently available. Try changing node !!! ".format(node))
    print("List of available nodes: {}".format(node_list))

#### Run Benchmark tool application with Intel® HD Graphics 530 GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870</a> edge node with an <a 
    href="https://devcloud.intel.com/edge/devices/intel-hd-530-gpu/">Intel® HD Graphics 530 GPU</a>. The inference workload will run on the GPU.

In [ ]:
node = 'idc001skl'
if node in node_list:
    print("Submitting a job to an edge compute node with an Intel Core CPU and an Intel GPU...")
    for model in models_fp16:
        benchmarks={}
        try: 
            # Submit job to the queue
            job_id_gpu = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/gpu/{model} GPU async {model} FP16"        
            benchmarks["Intel® HD Graphics 530 GPU"] = wait_for_job_to_finish(job_id_gpu,verbose,model)
            output[model].update(benchmarks)
        except Exception as e:
            print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
            print("Error: ", e)

    for model in models_fp32:
        benchmarks={}
        try: 
            # Submit job to the queue
            job_id_gpu = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/gpu/{model} GPU async {model} FP32"        
            benchmarks["Intel® HD Graphics 530 GPU"] = wait_for_job_to_finish(job_id_gpu,verbose,model)
            output[model].update(benchmarks)
        except Exception as e:
            print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
            print("Error: ", e)

else:
    print(" {} not currently available. Try changing node !!! ".format(node))
    print("List of available nodes: {}".format(node_list))

#### Run Benchmark tool application with Intel® Neural Compute Stick 2
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870</a> edge node with an <a 
    href="https://devcloud.intel.com/edge/devices/intel-movidius-myriadx-vpu-1/">Intel® Neural Compute Stick 2 VPU</a>. The inference workload will run on the NCS2.

In [ ]:
node = 'idc004nc2'
if node in node_list:
    if len(models_fp16) != 0:
        print("Submitting job to an edge compute node with Intel NCS2...")
        for model in models_fp16:
            benchmarks={}
            try:
                # Submit job to the queue
                job_id_ncs2 = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/ncs2/{model} MYRIAD async {model} FP16"    
                benchmarks["Intel® Neural Compute Stick 2"] = wait_for_job_to_finish(job_id_ncs2,verbose,model)
                output[model].update(benchmarks)
            except Exception as e:
                print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
                print("Error: ", e)
    else:
        print("FP32 models are incompatible with Intel NCS2...")
else:
    print(" {} not currently available. Try changing node !!! ".format(node))
    print("List of available nodes: {}".format(node_list))

#### Run Benchmark tool application with Intel® Movidius™ Myriad™ X HDDL
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870</a> edge node with an <a 
    href="https://devcloud.intel.com/edge/devices/intel-movidius-myriadx-vpu-8/">Intel® Movidius™ Myriad™ X HDDL VPU</a>. The inference workload will run on the HDDL-R.

In [ ]:
node = 'idc002mx8'
if node in node_list:
    if len(models_fp16) != 0:
        print("Submitting a job to an edge compute node with Intel Movidius HDDL-R...")
        for model in models_fp16:
            benchmarks={}
            try:
                # Submit job to the queue
                job_id_hddlr = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/hddlr/{model} HDDL async {model} FP16" 
                benchmarks["Intel® Movidius™ Myriad™ X HDDL"] = wait_for_job_to_finish(job_id_hddlr,verbose,model)
                output[model].update(benchmarks)
            except Exception as e:
                print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
                print("Error: ", e)
    else:
        print("FP32 models are incompatible with Intel HDDL-R...")
else:
    print(" {} not currently available. Try changing node !!! ".format(node))
    print("List of available nodes: {}".format(node_list))

## Multi plugin
Now let's try [MULTI plugin](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_MULTI.html) with different combination of available Inference Engine devices.

Multi-Device plugin automatically assigns inference requests to available computational devices to execute the requests in parallel. Potential gains are as follows:

- Improved throughput that multiple devices can deliver (compared to single-device execution)
- More consistent performance, since the devices can now share the inference burden (so that if one device is becoming too busy, another device can take more of the load)

Notice that we use multi-device and the application logic doesn't have to be changed. Meaning you don't need to explicitly load the network to every device, create and balance the inference requests and so on.

####  Run Benchmark tool application with MULTI: Intel® Core™ CPU, Intel® HD Graphics 530 GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870</a> edge node with an <a 
    href="https://devcloud.intel.com/edge/devices/intel-core-i5-6500te-cpu/">Intel® Core™ i5 6500TE CPU</a> and <a 
    href="https://devcloud.intel.com/edge/devices/intel-hd-530-gpu/">Intel® HD Graphics 530 GPU</a>. The inference workload will run on the MULTI:CPU,GPU.

In [ ]:
node = 'idc001skl'
if node in node_list:
    if len(models_fp16) != 0:
        print("Submitting a job to an edge compute node with an CPU and GPU...")
        for model in models_fp16:
            benchmarks={}
            try:
                # Submit job to the queue
                job_id_cpu_gpu = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/cpu_gpu/{model} MULTI:CPU,GPU sync {model} FP16" 
                benchmarks["MULTI:Intel® Core™ CPU,Intel® HD Graphics 530 GPU"] = wait_for_job_to_finish(job_id_cpu_gpu,verbose,model)
                output[model].update(benchmarks)
            except Exception as e:
                print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
                print("Error: ", e)
    else:
        print("FP32 models are incompatible with Multi plugins...")
else:
    print(" {} not currently available. Try changing node !!! ".format(node)) 
    print("List of available nodes: {}".format(node_list))

#### Run Benchmark tool application with MULTI:Intel® Movidius™ Myriad™ X HDDL,Intel® Core™ CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870</a> edge node with an <a 
    href="https://devcloud.intel.com/edge/devices/intel-core-i5-6500te-cpu/">Intel® Core™ i5 6500TE CPU</a> and <a 
    href="https://devcloud.intel.com/edge/devices/intel-movidius-myriadx-vpu-8/">Intel® Movidius™ Myriad™ X HDDL VPU</a>. The inference workload will run on the MULTI:HDDL,CPU.

In [ ]:
node = 'idc002mx8'
if node in node_list:
    if len(models_fp16) != 0:
        print("Submitting a job to an edge compute node with Intel CPU and Intel Movidius HDDL-R...")
        for model in models_fp16:
            benchmarks={}
            try:
                # Submit job to the queue
                job_id_cpu_hddl = !qsub benchmark_app_job.sh -l nodes=1:{node} -F "results/cpu_hddl/{model} MULTI:HDDL,CPU async {model} FP16" 
                benchmarks["MULTI:Intel® Movidius™ Myriad™ X HDDL,Intel® Core™ CPU"] = wait_for_job_to_finish(job_id_cpu_hddl,verbose,model)
                output[model].update(benchmarks)
            except Exception as e:
                print("\n===Failed to perform benchmarking for {} model !===\n".format(model))
                print("Error: ", e)
    else:
        print("FP32 models are incompatible with Multi plugins...")
else:
    print(" {} not currently available. Try changing node !!! ".format(node))  
    print("List of available nodes: {}".format(node_list))

## Benchmark Report

The running time of each inference task is recorded in output dictionary. Run the cell below to plot the results of all jobs side-by-side.


### Defining Function to Plot Bar Chart


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects

%matplotlib inline

def plot_benchmarks(model,metric):
    
    latency = {}
    no_number = False
    for device in output[model]:
        if isinstance(output[model][device][metric], str):
            no_number = True
        else:
            latency[device] = output[model][device][metric]    
        
    if not no_number:
        plt.figure(figsize=(18,8))
        plt.bar(*zip(*latency.items()));
        plt.xticks(fontsize=14);
        plt.yticks(fontsize=18);
        plt.ylabel(metric, fontsize=20);

        rects = plt.gca().patches

        # Make some labels.
        labels = ["{:,.2f}".format(i) for i in latency.values()]

        for rect, label in zip(rects, labels):
            height = rect.get_height()
            plt.gca().text(rect.get_x() + rect.get_width() / 2, height/2.0, label,
                    ha="center", va="bottom", fontsize=20, color="white", path_effects=[PathEffects.withStroke(linewidth=2, foreground="black")])
            
    else:
        print("ERROR: Field '{}' has text strings. Can't plot it.".format(metric))
        

In [ ]:
from ipywidgets import interact
import ipywidgets as widgets

In [ ]:
key = lambda out:out[next(iter(out))]

### Preparing Benchmark Results Tabular Format

In [ ]:
import pandas as pd
from IPython.display import display, HTML
import json

devices_list = ['Intel® Neural Compute Stick 2','Intel® Core™ CPU','Intel® HD Graphics 530 GPU','MULTI:Intel® Core™ CPU,Intel® HD Graphics 530 GPU','Intel® Movidius™ Myriad™ X HDDL','MULTI:Intel® Movidius™ Myriad™ X HDDL,Intel® Core™ CPU','Intel® Xeon® CPU']

link_file = open('device_link.json')
data = json.load(link_file)
df ={}

for model in output:
    print("\n {} \n".format(model))
    throughput = 0.0
    for device in output[model]:
        if (output[model][device]['Throughput (FPS)'] > throughput):
            throughput= output[model][device]['Throughput (FPS)']
            best_device = device
            link = data[device]
    display(HTML('<b>Model Name</b>: {}'.format(model)))
    display(HTML('<b>Best Device</b>(Based on Throughput) : {}'.format(best_device)))
    display(HTML('<b>Buy Now</b>: <a href=" {}">{}</a>'.format(link,link)))
    display(HTML('<u>Benchmark Results</u>'))
    df[model] = pd.DataFrame.from_dict(output[model], orient='index')
    if model in models_fp16:
        df[model]['Model Precision'] = 'FP16'
    elif model in models_fp32:
        df[model]['Model Precision'] = 'FP32'
    df[model] = df[model].reindex(devices_list).dropna(axis=0)
    display(HTML(df[model].to_html()))
link_file.close()

### Ploting Bar chart

Choose Model name and metrics 

In [ ]:
try:
    interact(plot_benchmarks, model=output.keys(),metric=key(output[next(iter(output))]).keys());
except :
    print("An error occured !")

## Email Notification

### Creating Notebook URL

In [ ]:
import subprocess

user_info = subprocess.check_output("pwd")
user_info = user_info[1:-1]
user_info = user_info.decode('utf-8')
user_info = user_info.split('/')
user = user_info[1]
path = ""
for i in range(2,len(user_info)):
    path = path + "/" + user_info[i]
notebook_url = "https://jupyter.edge.devcloud.intel.com/user/{}/notebooks{}/BenchmarkApp.ipynb".format(user,path)


### Sending Email Notification

In [ ]:
import boto3
from botocore.exceptions import ClientError

# The subject line for the email.
SUBJECT = "Message from Intel® DevCloud for the Edge"

# The email body for recipients with non-HTML email clients.
BODY_TEXT = ("Completed benchmarking of your models. Please visit your Intel® DevCloud Notebook. "
            )
            
# The HTML body of the email.
BODY_HTML = """<html>
<head></head>
<body>
  <h3> Message from Intel DevCloud </h3>
  <p>Completed benchmarking of your models. Please visit your <a href={}>Intel® DevCloud Notebook</a>.</p>
</body>
</html>
            """.format(notebook_url)         

# The character encoding for the email.
CHARSET = "UTF-8"

# Create a new SES resource and specify a region.
client = boto3.client('ses',region_name=AWS_REGION,aws_access_key_id=ACCESS_KEY_ID, aws_secret_access_key=SECRET_ACCESS_KEY)

# Try to send the email.
try:
    #Provide the contents of the email.
    response = client.send_email(
        Destination={
            'ToAddresses': [
                RECIPIENT,
            ],
        },
        Message={
            'Body': {
                'Html': {
                    'Charset': CHARSET,
                    'Data': BODY_HTML,
                },
                'Text': {
                    'Charset': CHARSET,
                    'Data': BODY_TEXT,
                },
            },
            'Subject': {
                'Charset': CHARSET,
                'Data': SUBJECT,
            },
        },
        Source=SENDER,
        # If you are not using a configuration set, comment or delete the
        # following line
        # ConfigurationSetName=CONFIGURATION_SET,
    )
# Display an error if something goes wrong.	
except ClientError as e:
    print(e.response['Error']['Message'])
else:
    print("Email sent! Message ID:"),
    print(response['MessageId'])

# Next steps
- [More Jupyter* Notebook Samples](https://devcloud.intel.com/edge/advanced/sample_applications/) - additional sample applications 
- [Jupyter* Notebook Tutorials](https://devcloud.intel.com/edge/get_started/tutorials) - sample application Jupyter* Notebook tutorials
- [Intel® Distribution of OpenVINO™ toolkit Main Page](https://software.intel.com/openvino-toolkit) - learn more about the tools and use of the Intel® Distribution of OpenVINO™ toolkit for implementing inference on the edge


# About this notebook

For technical support, please see the [Intel® DevCloud Forums](https://software.intel.com/en-us/forums/intel-devcloud-for-edge)

<p style=background-color:#0071C5;color:white;padding:0.5em;display:table-cell;width:100pc;vertical-align:middle>
<img style=float:right src="https://devcloud.intel.com/edge/static/images/svg/IDZ_logo.svg" alt="Intel DevCloud logo" width="150px"/>
<a style=color:white>Intel® DevCloud for the Edge</a><br>   
<a style=color:white href="#top">Top of Page</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Intel-DevCloud-for-the-Edge-Usage-Agreement.pdf">Usage Agreement (Intel)</a> | 
<a style=color:white href="https://devcloud.intel.com/edge/static/docs/terms/Colfax_Cloud_Service_Terms_v1.3.pdf">Service Terms (Colfax)</a>
</p>